In [1]:
from keras import backend as K
from keras.models import load_model
from keras.preprocessing import image
from keras.optimizers import Adam
from imageio import imread
import numpy as np
from matplotlib import pyplot as plt

from models.keras_ssd300 import ssd_300
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetections2 import DecodeDetections2
from keras_layers.keras_layer_L2Normalization import L2Normalization

from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

%matplotlib inline
import PIL.Image as Image

Using TensorFlow backend.


In [2]:
# Set the image size.
img_height = 300
img_width = 300

In [3]:
# TODO: Set the path to the `.h5` file of the model to be loaded.
model_path = '../model/model1.h5'

# We need to create an SSDLoss object in order to pass that to the model loader.
ssd_loss = SSDLoss(neg_pos_ratio=3, n_neg_min=0, alpha=1.0)

K.clear_session() # Clear previous models from memory.

model = load_model(model_path, custom_objects={'AnchorBoxes': AnchorBoxes,
                                               'L2Normalization': L2Normalization,
                                               'DecodeDetections': DecodeDetections,
                                               'compute_loss': ssd_loss.compute_loss})

In [4]:
import imageio
import pylab
orig_images = [] # Store the images here.
input_images = [] # Store resized versions of the images here.
filename = 'test1.mp4'
video = imageio.get_reader(filename)
listOfNumbers = [i for i in range(605)]
for number in listOfNumbers:
    image = video.get_data(number)
    image = Image.fromarray(image)
    orig_images.append(np.array(image))
    image = image.resize((300, 300), Image.ANTIALIAS)
# We'll only load one image in this example.
    input_images.append(np.array(image))
input_images = np.array(input_images)
orig_images = np.array(orig_images)

In [5]:
y_pred_decoded = np.load('y_pred_decoded.npy')

In [19]:
# Display the image and draw the predicted boxes onto it.

# Set the colors for the bounding boxes
colors = plt.cm.hsv(np.linspace(0, 1, 21)).tolist()
classes = ['background', 'plate']

fig = plt.figure(figsize = (20,12))
plt.imshow()
# for number in range(605):
#     fig = plt.figure(figsize=(20,12))
#     plt.imshow(orig_images[number])
#     current_axis = plt.gca()
#         # Transform the predicted bounding boxes for the 300x300 image to the original image dimensions.
#     if len(y_pred_decoded[number]) == 0:
#         image = Image.fromarray(orig_images[number])
#         image.save('videoImages/{}.jpg'.format(number))

#     for box in y_pred_decoded[number]:
#         # Transform the predicted bounding boxes for the 300x300 image to the original image dimensions.
#         xmin = box[2] * orig_images[number].shape[1] / img_width
#         ymin = box[3] * orig_images[number].shape[0] / img_height
#         xmax = box[4] * orig_images[number].shape[1] / img_width
#         ymax = box[5] * orig_images[number].shape[0] / img_height
#         color = colors[int(box[0])]
#         label = '{}: {:.2f}'.format(classes[int(box[0])], box[1])
#         current_axis.add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, color=color, fill=False, linewidth=2))  
#         current_axis.text(xmin, ymin, label, size='x-large', color='white', bbox={'facecolor':color, 'alpha':1.0})
#         current_axis.axis('off')
#         fig.savefig('videoImages/{}.jpg'.format(number), bbox_inches = 'tight', pad_inches = 0)
#     plt.close('all')

In [24]:
for number in range(605):
    image = Image.open('videoImages/{}.jpg'.format(number))
    image = image.crop((26, 3, 480, 655))
    image.save('videoImages/{}.jpg'.format(number))

In [26]:
import cv2

images = []
for i in range(605):
    images.append(cv2.imread('videoImages/{}.jpg'.format(i)))
    
height, width, layers = images[1].shape

video=cv2.VideoWriter('videoTest.avi', -1, 20, (width, height))

for j in range(605):
    video.write(images[j])
    
cv2.destroyAllWindows()
video.release()